## Load Data

In [ ]:
# Load emission data 
import pandas as pd
emission_data = pd.read_excel('emission_data.xlsx')

emission_data['Date_and_Time'] = pd.to_datetime(emission_data['Date_and_Time'])
emission_data = emission_data.drop(columns=['Station_Name', 'Latitude', 'Longitude', 'HC', 'Temperature', 'Humidity', 'Rainfall', 'Wind_Speed',
       'Wind_Direction', 'Global_Radiation']).dropna()



In [ ]:
# Load Traffic Data
# Load traffic data
traffic_data = pd.read_excel('traffic_data.xlsx')
traffic_data['Date_and_Time'] = pd.to_datetime(traffic_data['Date_and_Time'])


## Preprocessing

### Data integration

In [ ]:
# Merge traffic emission data
traffic_emission_data = pd.merge(traffic_data, emission_data, on='Date_and_Time', how='left')

traffic_emission_data

### Remove missing value

In [ ]:
traffic_emission_data.dropna(inplace=True)

### Outliers Handling

In [ ]:
# Show data distribution (box plot)
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Define the variables
# variables_plot = ['Total_Motorcycle', 
#                   'Total_Car', 
#                   'Total_BusTruck', 
#                   'Total_Total', 
#                   'Total_Velocity', 
#                   'Total_PCE', 
#                   'Total_VCRatio', 
#                   'PM10']

variables_plot = traffic_emission_data.columns[1:]



# Plot box plots
plt.figure(figsize=(10, 40))
for i, var in enumerate(variables_plot):
    plt.subplot(9, 3, i+1)
    sns.boxplot(data=traffic_emission_data, y=var)
    plt.title(f'Box Plot of {var}')
plt.tight_layout()
plt.show()


In [ ]:
# define function to remove outliers, more first
import pandas as pd

def remove_outliers(df, variables):
    """
    Removes outliers from specified variables in the DataFrame using the IQR method.
    
    Parameters:
        df (DataFrame): The DataFrame to clean.
        variables (list): A list of variable names from which to remove outliers.
    
    Returns:
        DataFrame: A DataFrame with outliers removed from the specified variables.
    """
    clean_df = df.copy()
    for var in variables:
        # Calculate Q1 (25th percentile) and Q3 (75th percentile)
        Q1 = clean_df[var].quantile(0.25)
        Q3 = clean_df[var].quantile(0.75)
        IQR = Q3 - Q1

        # Define bounds for outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Filter out outliers
        clean_df = clean_df[(clean_df[var] >= lower_bound) & (clean_df[var] <= upper_bound)]
    
    return clean_df


from sklearn.preprocessing import MinMaxScaler
def count_and_sort_outliers(data, variables):
    # Normalizing the data
    scaler = MinMaxScaler()
    data_scaled = pd.DataFrame(scaler.fit_transform(data[variables]), columns=variables)
    
    outlier_counts = {}
    
    # Counting outliers for each variable
    for variable in variables:
        Q1 = data_scaled[variable].quantile(0.25)
        Q3 = data_scaled[variable].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = data_scaled[(data_scaled[variable] < lower_bound) | (data_scaled[variable] > upper_bound)]
        outlier_counts[variable] = len(outliers)
    
    # Sorting variables by number of outliers
    sorted_variables = sorted(outlier_counts, key=outlier_counts.get, reverse=True)
    
    return sorted_variables

In [ ]:
# Define traffic variables
traffic_vars = ['Total_Motorcycle', 'Total_Car', 'Total_BusTruck', 'Total_Total', 'Total_PCE', 'Total_Velocity', 'Total_VCRatio']

# Define emission variables
emission_vars = ['PM10', 'PM2_5', 'SO2', 'CO', 'O3', 'NO2']
# emission_vars = ['PM10']

# Sort and Clean outliers
sorted_vars = count_and_sort_outliers(traffic_emission_data, traffic_vars+emission_vars)
data_clean = remove_outliers(traffic_emission_data, sorted_vars)
data_clean

### Feature selection

In [ ]:
data = data_clean[['Date_and_Time',
                   'Total_Motorcycle',
                   'Total_Car',
                   'Total_BusTruck',
                   'Total_Total',
                   'Total_PCE',
                   'Total_Velocity',
                   'Total_VCRatio',
                   'PM10']]

## EDA

In [ ]:
data.shape

In [ ]:
data.describe()

### Data distribution

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Define the variables
variables_plot = ['Total_Motorcycle', 
                  'Total_Car', 
                  'Total_BusTruck', 
                  'Total_Total', 
                  'Total_Velocity', 
                  'Total_PCE', 
                  'Total_VCRatio', 
                  'PM10']



# Plot histograms
plt.figure(figsize=(18, 12))
for i, var in enumerate(variables_plot):
    plt.subplot(3, 3, i+1)
    sns.histplot(data=data, x=var, kde=True)
    plt.title(f'Histogram of {var}')
plt.tight_layout()
plt.show()

# Plot box plots
plt.figure(figsize=(18, 12))
for i, var in enumerate(variables_plot):
    plt.subplot(3, 3, i+1)
    sns.boxplot(data=data, y=var)
    plt.title(f'Box Plot of {var}')
plt.tight_layout()
plt.show()


### Trend

In [ ]:
# Extracting features from 'Date_and_Time'
data['Hour'] = data['Date_and_Time'].dt.hour
data['Day'] = data['Date_and_Time'].dt.day
data['Month'] = data['Date_and_Time'].dt.month
data['Weekday'] = data['Date_and_Time'].dt.weekday

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


# Group by Hour to get the average values for each hour
hourly_data = data.groupby('Hour').mean().reset_index()

# Define traffic and PM emission variables
traffic_vars = [
    'Total_Motorcycle', 
    'Total_Car', 
    'Total_BusTruck',
    'Total_Total', 
    'Total_Velocity', 
    'Total_PCE', 
    'Total_VCRatio'	
    ]
pm_vars = ['PM10']

# Normalize the data
scaler = MinMaxScaler()
hourly_data[traffic_vars + pm_vars] = scaler.fit_transform(hourly_data[traffic_vars + pm_vars])

# Plotting hourly trends for traffic and PM emission variables
plt.figure(figsize=(12, 6))
for var in traffic_vars:
    sns.lineplot(data=hourly_data, x='Hour', y=var, label=var, ci=None)
for var in pm_vars:
    sns.lineplot(data=hourly_data, x='Hour', y=var, label=var, linestyle='--', ci=None)
plt.title('Normalized Hourly Trends of Traffic and PM Emission Levels')
plt.xlabel('Hour')
plt.ylabel('Normalized Value')
plt.legend(loc='upper right')
plt.grid(True)
plt.xticks(hourly_data['Hour'].unique())  # Only show hours that have data
plt.show()


### Correlation

In [ ]:
# Calculate the correlation matrix for the selected variables
correlation_matrix = data[traffic_vars + pm_vars].corr()

# Plot the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation between Traffic Variables and PM10 Emission Levels')
plt.xlabel('Variables')
plt.ylabel('Variables')
plt.show()

## Regression Model

### Prepare the data

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Load dataset
selected_column = ['Date_and_Time',	'Total_Motorcycle',	'Total_Car',	'Total_BusTruck',	'Total_Total',	'Total_PCE',	
                 'Total_Velocity',	'Total_VCRatio',	'PM10'	]
# selected_column = ['Date_and_Time',	'Total_Motorcycle',	'Total_Car',	'Total_BusTruck',	'PM10'	]
data_model = data[selected_column].copy()



data_model


### Model Comparation with k-Fold validation

In [ ]:
X = data_model.drop(columns=['Date_and_Time', 'PM10'])
y = data_model['PM10']

# Define models with pipelines that include scaling
models = {
    'SVM': Pipeline([('scaler', StandardScaler()), ('svm', SVR())]),
    'Random Forest': RandomForestRegressor(),  # No scaling needed for Random Forest
    'MLP': Pipeline([('scaler', StandardScaler()), ('mlp', MLPRegressor(max_iter=1000))])
}

# Set up k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Function to evaluate models
def evaluate_model_r2(model, X, y, kf):
    scores = cross_val_score(model, X, y, scoring='r2', cv=kf)
    return scores  # Return R² scores

# Dictionary to hold the results
results = {}

# Evaluate each model using k-fold cross-validation
for model_name, model in models.items():
    r2_scores = evaluate_model_r2(model, X, y, kf)
    results[model_name] = r2_scores
    print(f'{model_name} R²: {r2_scores.mean():.4f} ± {r2_scores.std():.4f}')

# Convert results to a DataFrame for better readability
results_df = pd.DataFrame(results)

# Display the results
print("\nModel Selection Results (R²):")
print(results_df)

In [ ]:
# Extracting features from 'Date_and_Time'
data_model['Hour'] = data['Date_and_Time'].dt.hour
data_model['Day'] = data['Date_and_Time'].dt.day
data_model['Month'] = data['Date_and_Time'].dt.month
data_model['Weekday'] = data['Date_and_Time'].dt.weekday

X = data_model.drop(columns=['Date_and_Time', 'PM10'])
y = data_model['PM10']

# Define models with pipelines that include scaling
models = {
    'SVM': Pipeline([('scaler', StandardScaler()), ('svm', SVR())]),
    'Random Forest': RandomForestRegressor(),  # No scaling needed for Random Forest
    'MLP': Pipeline([('scaler', StandardScaler()), ('mlp', MLPRegressor(max_iter=1000))])
}

# Set up k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Function to evaluate models
def evaluate_model_r2(model, X, y, kf):
    scores = cross_val_score(model, X, y, scoring='r2', cv=kf)
    return scores  # Return R² scores

# Dictionary to hold the results
results = {}

# Evaluate each model using k-fold cross-validation
for model_name, model in models.items():
    r2_scores = evaluate_model_r2(model, X, y, kf)
    results[model_name] = r2_scores
    print(f'{model_name} R²: {r2_scores.mean():.4f} ± {r2_scores.std():.4f}')

# Convert results to a DataFrame for better readability
results_df = pd.DataFrame(results)

# Display the results
print("\nModel Selection Results (R²):")
print(results_df)

### Selected RF Model

In [ ]:
# Initial RF Model

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

X = data_model.drop(columns=['Date_and_Time', 'PM10'])
y = data_model['PM10']

# Assume X and y are already defined as your features and target variable
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define Random Forest model
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)




In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Get feature importances
feature_importances = rf.feature_importances_

# Create a DataFrame for better visualization
features = X.columns  # assuming X is a pandas DataFrame
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': feature_importances
})

# Sort the DataFrame by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Print the feature importances
print(importance_df)

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Feature Importances in Random Forest')
plt.gca().invert_yaxis()  # To display the highest importance at the top
plt.show()

# Predictions
y_pred_train = rf.predict(X_train)
y_pred_test = rf.predict(X_test)

# Evaluation metrics
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

n = X_test.shape[0]
p = X_test.shape[1]
adjusted_r2 = 1 - (1 - r2_test) * (n - 1) / (n - p - 1)

mae = mean_absolute_error(y_test, y_pred_test)
mse = mean_squared_error(y_test, y_pred_test)
rmse = np.sqrt(mse)

# Mean Forecast Error (MFE) and Mean Relative Error (MRE)
mfe = np.mean(y_test - y_pred_test)
# mre = np.mean(np.abs((y_test - y_pred_test) / y_test))

# Filter out zero values in y_test to avoid division by zero
non_zero_indices = y_test != 0
y_test_non_zero = y_test[non_zero_indices]
y_pred_test_non_zero = y_pred_test[non_zero_indices]
mre = np.mean(np.abs((y_test_non_zero - y_pred_test_non_zero) / y_test_non_zero))



# Print evaluation metrics
print(f'R² (train): {r2_train:.4f}')
print(f'R² (test): {r2_test:.4f}')
print(f'Adjusted R² (test): {adjusted_r2:.4f}')
print(f'MAE: {mae:.4f}')
print(f'MSE: {mse:.4f}')
print(f'RMSE: {rmse:.4f}')
print(f'MFE: {mfe:.4f}')
print(f'MRE: {mre:.4f}')

# Plot actual vs predicted
plt.figure(figsize=(10, 5))
plt.scatter(y_test, y_pred_test, edgecolors=(0, 0, 0))
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'k--', lw=3)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted')
plt.show()

# Plot residuals vs predicted
residuals = y_test - y_pred_test
plt.figure(figsize=(10, 5))
plt.scatter(y_pred_test, residuals, edgecolors=(0, 0, 0))
plt.hlines(y=0, xmin=min(y_pred_test), xmax=max(y_pred_test), colors='r', linestyles='dashed')
plt.xlabel('Predicted')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted')
plt.show()


### Feature Reduction

In [ ]:
# Define a threshold for feature selection
threshold = 0.03  # This threshold can be adjusted based on your specific needs

# Select features above the threshold
selected_features = importance_df[importance_df['Importance'] > threshold]['Feature']
X_train_reduced = X_train[selected_features]
X_test_reduced = X_test[selected_features]


# Define Random Forest model
rf_reduced = RandomForestRegressor(random_state=42)
rf_reduced.fit(X_train_reduced, y_train)




In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Get feature importances
feature_importances_reduced = rf_reduced.feature_importances_

# Create a DataFrame for better visualization
importance_df_reduced = pd.DataFrame({
    'Feature': selected_features,
    'Importance': feature_importances_reduced
})

# Sort the DataFrame by importance
importance_df_reduced = importance_df_reduced.sort_values(by='Importance', ascending=False)

# Print the feature importances
print(importance_df_reduced)

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.barh(importance_df_reduced['Feature'], importance_df_reduced['Importance'])
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Feature Importances in Random Forest (Reduced Features)')
plt.gca().invert_yaxis()  # To display the highest importance at the top
plt.show()

# Predictions for reduced model
y_pred_train_reduced = rf_reduced.predict(X_train_reduced)
y_pred_test_reduced = rf_reduced.predict(X_test_reduced)

# Evaluation metrics for reduced model
r2_train_reduced = r2_score(y_train, y_pred_train_reduced)
r2_test_reduced = r2_score(y_test, y_pred_test_reduced)

n = X_test_reduced.shape[0]
p = X_test_reduced.shape[1]
adjusted_r2_reduced = 1 - (1 - r2_test_reduced) * (n - 1) / (n - p - 1)

mae_reduced = mean_absolute_error(y_test, y_pred_test_reduced)
mse_reduced = mean_squared_error(y_test, y_pred_test_reduced)
rmse_reduced = np.sqrt(mse_reduced)

# Mean Forecast Error (MFE) and Mean Relative Error (MRE) for reduced model
mfe_reduced = np.mean(y_test - y_pred_test_reduced)
# mre_reduced = np.mean(np.abs((y_test - y_pred_test_reduced) / y_test))

# Filter out zero values in y_test to avoid division by zero
non_zero_indices = y_test != 0
y_test_non_zero = y_test[non_zero_indices]
y_pred_test_reduced_non_zero = y_pred_test_reduced[non_zero_indices]
mre_reduced = np.mean(np.abs((y_test_non_zero - y_pred_test_reduced_non_zero) / y_test_non_zero))



# Print evaluation metrics for reduced model
print(f'R² (train): {r2_train_reduced:.4f}')
print(f'R² (test): {r2_test_reduced:.4f}')
print(f'Adjusted R² (test): {adjusted_r2_reduced:.4f}')
print(f'MAE: {mae_reduced:.4f}')
print(f'MSE: {mse_reduced:.4f}')
print(f'RMSE: {rmse_reduced:.4f}')
print(f'MFE: {mfe_reduced:.4f}')
print(f'MRE: {mre_reduced:.4f}')

# Plot actual vs predicted for reduced model
plt.figure(figsize=(10, 5))
plt.scatter(y_test, y_pred_test_reduced, edgecolors=(0, 0, 0))
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'k--', lw=3)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted (Reduced Features)')
plt.show()

# Plot residuals vs predicted for reduced model
residuals_reduced = y_test - y_pred_test_reduced
plt.figure(figsize=(10, 5))
plt.scatter(y_pred_test_reduced, residuals_reduced, edgecolors=(0, 0, 0))
plt.hlines(y=0, xmin=min(y_pred_test_reduced), xmax=max(y_pred_test_reduced), colors='r', linestyles='dashed')
plt.xlabel('Predicted')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted (Reduced Features)')
plt.show()

### Hyperparameter Tuning

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score



# Hyperparameter space for GridSearchCV
param_grid1 = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Perform GridSearchCV
grid_search1 = GridSearchCV(estimator=rf, param_grid=param_grid1,
                           cv=3, n_jobs=-1, verbose=2)
grid_search1.fit(X_train_reduced, y_train)

# Best parameters from GridSearchCV
best_params_grid1 = grid_search1.best_params_
best_model_grid1 = grid_search1.best_estimator_

# Print the best parameters
print("Best parameters from GridSearchCV: ", best_params_grid1)

# Evaluate the model on the test set
test_score_grid1 = best_model_grid1.score(X_test_reduced, y_test)
print("Test set score with best parameters from GridSearchCV: ", test_score_grid1)

# Predictions for reduced model
y_pred_train_grid1 = best_model_grid1.predict(X_train_reduced)
y_pred_test_grid1 = best_model_grid1.predict(X_test_reduced)

# Evaluation metrics for GridSearchCV model
r2_train_grid1 = r2_score(y_train, y_pred_train_grid1)
r2_test_grid1 = r2_score(y_test, y_pred_test_grid1)

print(f'R² (train): {r2_train_grid1:.4f}')
print(f'R² (test): {r2_test_grid1:.4f}')

In [ ]:
# Hyperparameter space for second GridSearchCV
param_grid2 = {
    'bootstrap': [True, False],
    'criterion': ['squared_error', 'absolute_error'],
    'oob_score': [True, False],
    'n_estimators': [best_params_grid1['n_estimators']],
    'max_depth': [best_params_grid1['max_depth']],
    'min_samples_split': [best_params_grid1['min_samples_split']],
    'min_samples_leaf': [best_params_grid1['min_samples_leaf']],
    'max_features': [best_params_grid1['max_features']]
}

# Define Random Forest model with best parameters from first GridSearchCV
rf_tuned = RandomForestRegressor(
    random_state=42,
    n_estimators=best_params_grid1['n_estimators'],
    max_depth=best_params_grid1['max_depth'],
    min_samples_split=best_params_grid1['min_samples_split'],
    min_samples_leaf=best_params_grid1['min_samples_leaf'],
    max_features=best_params_grid1['max_features']
)

# Perform GridSearchCV again with additional parameters
grid_search2 = GridSearchCV(estimator=rf_tuned, param_grid=param_grid2,
                            cv=3, n_jobs=-1, verbose=2)
grid_search2.fit(X_train_reduced, y_train)

# Best parameters from second GridSearchCV
best_params_grid2 = grid_search2.best_params_
best_model_grid2 = grid_search2.best_estimator_

# Print the best parameters
print("Best parameters from second GridSearchCV: ", best_params_grid2)

# Evaluate the model on the test set
test_score_grid2 = best_model_grid2.score(X_test_reduced, y_test)
print("Test set score with best parameters from second GridSearchCV: ", test_score_grid2)

# Predictions for reduced model
y_pred_train_grid2 = best_model_grid2.predict(X_train_reduced)
y_pred_test_grid2 = best_model_grid2.predict(X_test_reduced)

# Evaluation metrics for second GridSearchCV model
r2_train_grid2 = r2_score(y_train, y_pred_train_grid2)
r2_test_grid2 = r2_score(y_test, y_pred_test_grid2)

print(f'R² (train): {r2_train_grid2:.4f}')
print(f'R² (test): {r2_test_grid2:.4f}')


In [ ]:
# Define Random Forest model with best parameters from the second GridSearchCV
rf_tuned = RandomForestRegressor(
    random_state=42,
    n_estimators=best_params_grid2['n_estimators'],
    max_depth=best_params_grid2['max_depth'],
    min_samples_split=best_params_grid2['min_samples_split'],
    min_samples_leaf=best_params_grid2['min_samples_leaf'],
    max_features=best_params_grid2['max_features'],
    bootstrap=best_params_grid2['bootstrap'],
    criterion=best_params_grid2['criterion'],
    oob_score=best_params_grid2['oob_score']
)

# Train the model with reduced features and best parameters
rf_tuned.fit(X_train_reduced, y_train)

# Get feature importances
feature_importances_tuned = rf_tuned.feature_importances_

# Create a DataFrame for better visualization
importance_df_tuned = pd.DataFrame({
    'Feature': selected_features,
    'Importance': feature_importances_tuned
})

# Sort the DataFrame by importance
importance_df_tuned = importance_df_tuned.sort_values(by='Importance', ascending=False)

# Print the feature importances
print(importance_df_tuned)

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.barh(importance_df_tuned['Feature'], importance_df_tuned['Importance'])
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Feature Importances in Random Forest (Tuned Hyperparameter)')
plt.gca().invert_yaxis()  # To display the highest importance at the top
plt.show()

# Predictions for reduced model
y_pred_train_tuned = rf_tuned.predict(X_train_reduced)
y_pred_test_tuned = rf_tuned.predict(X_test_reduced)

# Evaluation metrics for reduced model
r2_train_tuned = r2_score(y_train, y_pred_train_tuned)
r2_test_tuned = r2_score(y_test, y_pred_test_tuned)

n = X_test_reduced.shape[0]
p = X_test_reduced.shape[1]
adjusted_r2_tuned = 1 - (1 - r2_test_tuned) * (n - 1) / (n - p - 1)

mae_tuned = mean_absolute_error(y_test, y_pred_test_tuned)
mse_tuned = mean_squared_error(y_test, y_pred_test_tuned)
rmse_tuned = np.sqrt(mse_tuned)

# Mean Forecast Error (MFE) and Mean Relative Error (MRE) for tuned model
mfe_tuned = np.mean(y_test - y_pred_test_tuned)

# Filter out zero values in y_test to avoid division by zero
non_zero_indices = y_test != 0
y_test_non_zero = y_test[non_zero_indices]
y_pred_test_tuned_non_zero = y_pred_test_tuned[non_zero_indices]
mre_tuned = np.mean(np.abs((y_test_non_zero - y_pred_test_tuned_non_zero) / y_test_non_zero))

# Print evaluation metrics for reduced model
print(f'R² (train): {r2_train_tuned:.4f}')
print(f'R² (test): {r2_test_tuned:.4f}')
print(f'Adjusted R² (test): {adjusted_r2_tuned:.4f}')
print(f'MAE: {mae_tuned:.4f}')
print(f'MSE: {mse_tuned:.4f}')
print(f'RMSE: {rmse_tuned:.4f}')
print(f'MFE: {mfe_tuned:.4f}')
print(f'MRE: {mre_tuned:.4f}')

# Plot actual vs predicted for reduced model
plt.figure(figsize=(10, 5))
plt.scatter(y_test, y_pred_test_tuned, edgecolors=(0, 0, 0))
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'k--', lw=3)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted (Tuned Hyperparameter)')
plt.show()

# Plot residuals vs predicted for tuned model
residuals_tuned = y_test - y_pred_test_tuned
plt.figure(figsize=(10, 5))
plt.scatter(y_pred_test_tuned, residuals_tuned, edgecolors=(0, 0, 0))
plt.hlines(y=0, xmin=min(y_pred_test_tuned), xmax=max(y_pred_test_tuned), colors='r', linestyles='dashed')
plt.xlabel('Predicted')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted (Tuned Hyperparameter)')
plt.show()

## Time-Series Model

### Prepare the data

In [ ]:
data_LSTM = data[['Date_and_Time','PM10']]
data_LSTM.set_index('Date_and_Time', inplace=True)

### Set the LSTM model

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import random

# Data preparation function for LSTM
def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(len(data)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(data)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = data[i:end_ix], data[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Parameters
n_steps = 24  # Number of time steps for each input
n_features = 1  # PM10 is a single feature



# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data_LSTM.values.reshape(-1, 1))

# Prepare the data
X, y = prepare_data(data_scaled, n_steps)

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

# Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
train_size = int(len(X) * 0.8)  # 80% for training
X_train, X_test = X[:train_size], X[train_size:]  # Ambil data awal untuk training
y_train, y_test = y[:train_size], y[train_size:]  # Sisakan data akhir untuk testing


# Reshape from [samples, timesteps] into [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))

# Create the LSTM model
model = Sequential()

# Add LSTM layers with dropout
for _ in range(6):  # 6 LSTM layers
    model.add(LSTM(units=30, activation='relu', return_sequences=True))  # 30 units per LSTM layer
    model.add(Dropout(0.2))  # Dropout rate of 0.2

# Adding a final LSTM layer without return_sequences
model.add(LSTM(units=30, activation='relu'))  
model.add(Dropout(0.2))

# Output layer
model.add(Dense(1))  # Assuming it's a regression problem with one output

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Fit the model
from tensorflow.keras.callbacks import EarlyStopping

# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping], shuffle=False)



### Training

In [ ]:

history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), verbose=1)

history.history.keys()

### Evaluation

In [ ]:
# Making predictions
predictions = model.predict(X_test)

# Rescale predictions back to the original scale
predictions_rescaled = scaler.inverse_transform(predictions)
y_test_rescaled = scaler.inverse_transform(y_test)

# Calculate performance metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
rmse = mean_squared_error(y_test_rescaled, predictions_rescaled, squared=False)
mae = mean_absolute_error(y_test_rescaled, predictions_rescaled)
r2 = r2_score(y_test_rescaled, predictions_rescaled)

print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'R2: {r2}')

# Plot the loss
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()





plt.figure(figsize=(19, 6))
plt.plot(y_test_rescaled, label='Actual', color='blue', marker='o')
plt.plot(predictions_rescaled, label='Predicted', color='red', linestyle='--', marker='x')
plt.title('PM10 Actual vs Predicted')
plt.xlabel('Time')
plt.ylabel('PM10 Concentration')
plt.legend()
plt.show()